In [1]:
# Clone the repository and install the necessary packages
!git clone https://github.com/JakubMartinka/karlsruhe2024.git
!pip3 install mlatom
!pip3 install mkl
!pip3 install mkl-service
!pip3 install py3Dmol
!pip3 install geometric
!pip3 install pyscf==2.4.0

Cloning into 'karlsruhe2024'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 110 (delta 28), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 30.16 MiB | 12.71 MiB/s, done.
Resolving deltas: 100% (28/28), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.0/385.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for geometric: filename=geometric-1.0.2-py3-none-any.whl size=397151 sha256=373ee7349e467c4547ef3eab345dcd1f05d680907d511a2536663be299d4d04a
  Stored in directory: /root/.cache/pip/wheels/a6/9a/7c/

In [2]:
import mlatom as ml

In [3]:
# get initial structure
h2 = ml.molecule.from_xyz_string('''2

H 0 0 0
H 0 0 0.7
''')

In [4]:
# define the reference method
dft = ml.models.methods(method='B3LYP/6-31G*', program='pyscf')

In [11]:
# optimize geometry and calculate frequency
#optmol = ml.optimize_geometry(
#    model=dft,
#    program='geometric',
#    initial_molecule=h2
#    ).optimized_molecule
#freq = ml.freq(model=dft,vmolecule=optmol)
optmol = ml.molecule.load('karlsruhe2024/materials/h2_freq.json')

In [12]:
# settings for sampling points during AL loop
sampler_kwargs = {
    'initcond_sampler':'wigner',
    'initcond_sampler_kwargs':{
        'molecule':optmol,
        'number_of_initial_conditions':100,
        'initial_temperature':300},
    'maximum_propagation_time':100.0,
    'time_step':0.5,
    'uq_threshold':0.000001,
}

In [13]:
# start active learning
ml.al(molecule=optmol,
      ml_model='KREG',
      sampler = 'md',
      sampler_kwargs=sampler_kwargs,
      min_new_points=5,
      reference_method=dft)

 Initial points sampling: Use cross validation
 Initial points sampling: Number of CV splits = 5
Start initial points sampling...
 Initial points sampling: initial points RMSE threshold not found, fit learning curve instead
 Initial points sampling: Train on energies only


AttributeError: 'methods' object has no attribute 'interface'

In [ ]:
# check the final labeled database
db = ml.molecular_database.load(filename='labeled_db.json', format='json')
len(db)

In [ ]:
for imol in range(len(db)):
    print(db[imol].internuclear_distance(0, 1), db[imol].energy)

In [ ]:
db.view()